<a href="https://colab.research.google.com/github/liao771018/LINE-BOT/blob/main/LINE_BOT_%E6%89%80%E6%9C%89%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 寫程式的流程

# 安裝套件

# 引用套件

# 建置主程序

# 為主程序製作交互API接口

# 啟動主程序

# python語言最常見的一種http套件，叫做flask

In [ ]:
# 主程序

# flask 套件
from flask import Flask, request, abort
# line 套件
from linebot import (
    LineBotApi, WebhookHandler
)
# line 驗證消息的套件
from linebot.exceptions import (
    InvalidSignatureError
)
# line 的消息套件
from linebot.models import (
    MessageEvent, FollowEvent, TextMessage, TextSendMessage, ImageMessage
)

# 圖片下載與上傳專用
import urllib.request
import os

# 建立日誌紀錄設定檔
# https://googleapis.dev/python/logging/latest/stdlib-usage.html
import logging
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler

# 啟用log的客戶端
client = google.cloud.logging.Client()


# 建立line event log，用來記錄line event
# 會將 Log 紀錄到 ai_bot_event 檔內
bot_event_handler = CloudLoggingHandler(client,name="ai_bot_event")
bot_event_logger = logging.getLogger('ai_bot_event')
bot_event_logger.setLevel(logging.INFO)
bot_event_logger.addHandler(bot_event_handler)

# 準備 app
app = Flask(__name__)

# 註冊機器人
# 跟 line 溝通用的
# line_bot_api = LineBotApi('YOUR_CHANNEL_ACCESS_TOKEN')
# 接收 line 消息用的
# handler = WebhookHandler('YOUR_CHANNEL_SECRET')
line_bot_api = LineBotApi('YOUR_CHANNEL_ACCESS_TOKEN')
handler = WebhookHandler('YOUR_CHANNEL_SECRET')


# 設定 https 機器人訪問入口，給 line 傳消息進來用 
@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value  # LINE的加密簽章，不用變更
    signature = request.headers['X-Line-Signature']

    # get request body as text # 用戶傳來的內容
    body = request.get_data(as_text=True)
    print(body)
    # 消息整個交給bot_event_logger，請它傳回GCP
    bot_event_logger.info(body)
    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'
# 以上這段入口設定是固定的，不用更改


In [ ]:
# FollowEvent 追蹤
# 當 handler 收到用戶的追蹤時
from google.cloud import storage
from google.cloud import firestore

@handler.add(FollowEvent)
def handle_follow_event(event):

    # 用line_bot_api 向 line 取個資
    line_user_profile = line_bot_api.get_profile(event.source.user_id)

    # 跟 line 取回大頭照，並放置下載在本地端
    file_name = line_user_profile.user_id+'.jpg'
    urllib.request.urlretrieve(line_user_profile.picture_url, file_name)


    # 將取回的大頭照上傳到 Cloud Storage 以用戶 ID 命名的資料夾內 (upload_demo.py的程式碼)
    storage_client = storage.Client()
    bucket_name = "YOUR-BUCKET-NAME"
    destination_blob_name = f"{line_user_profile.user_id}/user_pic.png"
    source_file_name = file_name
    
    # 進行上傳
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

    
    # 把用戶的資料儲存到 Firestore 資料庫 (insert_data_to_firestore_demo.py的程式碼)
    # 設定用戶資料json
    user_dict={
        "user_id":line_user_profile.user_id,
        "picture_url": f"https://storage.googleapis.com/{bucket_name}/destination_blob_name",
        "display_name": line_user_profile.display_name,
        "status_message": line_user_profile.status_message
    }
    # 新增資料到 firestore
    db = firestore.Client()
    doc_ref = db.collection(u'line-user').document(user_dict.get("user_id"))
    doc_ref.set(user_dict)


    # 請line_bot_api 回傳文字消息
    # 可以傳歡迎的訊息
    line_bot_api.reply_message(
    event.reply_token,
    TextSendMessage(text="你的個資已被我取走了"))


In [ ]:
# 回應消息的準備
'''

消息素材準備

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)


''' 回應文字 '''
specific_text_message=TextSendMessage("官方建議，用戶以文字消息觸發特殊功能的時候，要加上特殊符號")


''' 回應圖片 '''
# 必須是網址，可以是先將圖片上傳到Cloud Storage
analyze_truth_image_message=ImageSendMessage(
      original_content_url='https://i.imgur.com/M8jnn9B.png',
      preview_image_url='https://i.imgur.com/LtCtgg0.jpg'
      )


''' 回應 TemplateSendMessage 按鍵範本 '''
# 可以到　LINE BOT DESIGNER 上面看

#引入所需要的消息與模板消息
from linebot.models import (
    MessageEvent, TemplateSendMessage , PostbackEvent
)

#引入按鍵模板
from linebot.models.template import(
    ButtonsTemplate
)


'''
alt_text: Line簡覽視窗所出現的說明文字 (LINE電腦版看到的文字)
template: 所使用的模板
ButtonsTemplate: 按鍵模板
    thumbnail_image_url: 展示圖片
    title: 標題
    text: 說明文字
    actions: 模板行為所使用的行為 (json檔格式)
    data: 觸發postback後用戶回傳值，可以對其做商業邏輯處理

'''
# 按鍵範本最多只能有四個
buttons_template_message = TemplateSendMessage(
    alt_text='Buttons template',
    template=ButtonsTemplate(
        title='更多幫助',
        text='請點擊下方按鈕獲得更多幫助',
        actions=[
          {
            "type": "postback",
            "label": "企業，查找商業結合方案",
            "text": "[::text:]尋找BD", # 以用戶身分傳出的文字
            "data": "Data1"      # 觸發 PostbackEvent 事件回傳的資料
          },
          {
            "type": "postback",
            "label": "開發者，尋求教學",
            "text": "[::text:]求助專家",
            "data": "Data2"
          },
          {
            "type": "uri",
            "label": "url scheme",
            "uri": "https://"
          },
          {
            "type": "uri",
            "label": "打電話",
            "uri": "tel://0212345678"
          }
        ],
  )
)
# tyep = {message:文字訊息, DateTimePick:時間按鍵, URI:外部連結, Postback:一個文字消息+一個PostbackEnet, Camera:開啟相機拍照, CameraRoll:選擇相片, Location:地理位置}



''' 回應 QuickReply 的 Button '''
# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)


# 創建QuickReplyButton 
# 有很多的小按鍵，按完後就會消失，出現後只有一次點擊機會，可以解決 button template 可以重覆點擊的問題


## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息",  # 按鍵顯示的文字
        text="text2"      # 用戶角度發送的文字
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",    # 隱藏資訊給用戶                   
        mode="date"     # 給用戶輸入的畫面是天數
    )
)

## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB, postbackQRB]
)

## QuickReply 通常會夾帶在 xxxxSendMessage 的 quick_reply 參數中
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
# 將上方設定好的回應方式製作成字典
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  # 回應上方設定好的文字
  "@123":specific_text_message,
  # 回應設定好的圖片
  "@分析的本質": analyze_truth_image_message,  
  # 回應設定好的按鍵範本
  "@more":buttons_template_message,
  # 回應設定好的 quick reply
  "@reply":quick_reply_text_send_message       
}

'''
確認按鈕  ConfirmTemplate
輪播訊息  ImageCarouselTemplate
'''

In [ ]:
# MessageEvent, TextMessage 文字消息
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    # 當用戶傳的消息有 @ 符號且不是最後一個， 則回應字典預設的內容
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        ) # event.mesage.text是用戶傳來的文字
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字")
        )

# 當 handler 收到用戶的文字消息的時候，回傳用戶講過的話
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    # 請line_bot_api回應，回應用戶講過的話
    line_bot_api.reply_message(
        event.reply_token,[
        TextSendMessage('A'),
        TextSendMessage('B'),
        TextSendMessage('C'),
        TextSendMessage('D')
        ]
        )

In [ ]:
# PostbackEvent 事件觸發
#用戶點擊 buttons_template_message 或 quick_reply 的 Postback 類型後，觸發 PostbackEvent，對回傳設定的data資料做相對應處理
@handler.add(PostbackEvent)
def handle_post_message(event):

    user_profile = line_bot_api.get_profile(event.source.user_id)

    # 把 Postback 回傳的 data 做分類 
    if (event.postback.data.find('Data1')== 0):                 # 這裡 == 0 看不懂
        # 寫入檔案
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):

        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextMessage(
                    text='請稍待，我們會派專家與您聯繫'
                )
            )
    else:
        pass

In [ ]:
# MessageEvent 圖片消息 ImageMesage
# 當 handler 收到用戶的圖片消息時
from linebot.models import ImageMessage

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
  
    # 請line_bot_api 取出照片
    image_blob = line_bot_api.get_message_content(event.message.id)
    temp_file_path = f"""{event.message.id}.png"""

    with open(temp_file_path, 'wb') as fd:
        for chunk in image_blob.iter_content():
            fd.write(chunk)

    # 上傳至cloud storage
    storage_client = storage.Client()
    bucket_name = "YOUR-BUCKET-NAME"
    destination_blob_name = f'{event.source.user_id}/image/{event.message.id}.png'
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(temp_file_path)

    # 未來放ai功能


    # 回應用戶
    line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(f"""圖片已上傳，請期待未來的AI服務！""")
        )
'''

影片消息，更改為 VideoMessage
音訊消息，更改為 AudioMessage
貼圖  StrickMessage

'''

In [ ]:
# 追加程式碼要在這行 app.run 的上方
# 自動運行伺服器在 8080 PORT
if __name__ == "__main__":
    app.run(host="0.0.0.0", port=int(os.environ.get("PORT", 8080)))
